In [1]:
import json
import os
import numpy as np

from subprocess      import call
from IPython.display import clear_output

from FELTOR_class    import FELTOR_Cpp

# Running FELTOR From Jupyter notebook with FELTOR_class

The FELTOR program is an efficient tool that allows us to run MHD simulations to study turbulence in Nuclear Fusion reactors.

To do this we use the program `convection_hpc`  allocated at `../FELTOR/`. This program needs and import file `input.json`, an example of it can be seen in the variable `data` and an empty output `output.nc`. 

This model, can be run two different models: __interchange model__ (IC) and __Hasegawa Wakatani model__ (HW) with and without average. To select one or the other model we need to set the variable `model` as `"HW"` or `"IC"` (effectively only the name `"HW"` makes a difference).

The first model is represented by the equations:

\begin{align}
 \nabla^2 \phi =  \omega, \quad \\
 \frac{\partial n}{\partial t}     =
    \{ n, \phi\}
  + \kappa \frac{\partial \phi}{\partial y}
  + \nu \nabla^2 n  \\
  \frac{\partial \omega}{\partial t} =
  \{ \omega, \phi\}
  - \kappa\frac{\partial n}{\partial y} +\nu\nabla^2\omega
\end{align}


For the Hasegawa - Wakatani model we have the following equations

\begin{align}
 \nabla^2 \phi =  \omega, \quad \\
 \frac{\partial n}{\partial t}     = - g \frac{\partial \phi}{\partial y} + \alpha (\tilde{\phi} - \tilde{n})
 + \{n, \phi\} + D \nabla^2 n \\
  \frac{\partial \omega}{\partial t} = \alpha ( \tilde{\phi} - \tilde{n}) + \{ \omega, \phi\}
  + \mu\nabla^2\omega
\end{align}

The variables $\tilde{\phi}$ and $\tilde{n}$ represents low frequency fluctuations. Their relation with $\phi$ and $n$ is given by

\begin{align}
\phi = \tilde{\phi} + <\phi> \\
n = \tilde{n} + <n>
\end{align}

where $<...>$ represents the averagive in the poloidal direction. For this reason, we can write the HW model equations as

\begin{align}
 \nabla^2 \phi =  \omega, \quad \\
 \frac{\partial n}{\partial t}     = - g \frac{\partial \phi}{\partial y} + \alpha [(\phi - <\phi>) - (n - <n>)]
 + \{n, \phi\} + D \nabla^2 n \\
  \frac{\partial \omega}{\partial t} = \alpha [(\phi - <\phi>) - (n - <n>)]  + \{ \omega, \phi\}
  + \mu\nabla^2\omega
\end{align}
    
    
In this case, we can run the model with and without the averages, which allows us to play with two different approximations. For that we need to set the variable `make_average` as true or flase (1 or 0).

In [2]:
data = {
        "model" : "HW",
        "make_average": 1,
        "n" :  3,
        "Nx" : 120,
        "Ny" : 120,
        "dt" : 0.01,
        "n_out"  : 3,
        "Nx_out" : 60,
        "Ny_out" : 60,
        "itstp"  : 10,
        "maxout" : 80,
        "eps_pol"   : 1e-6,
        "eps_time"  : 1e-10,
        "stages" : 3,
        "curvature"  : 0.015,
        "dens_prof"  : -0.015,
        "adiabatic"  : 0.0001,
        "nu_perp"  : 5e-3,
        "amplitude"  : 0.5,
        "sigma"  : 10,
        "posX"  : 0.5,
        "posY"  : 0.5,
        "lx"  : 200,
        "ly"  : 200,
        "bc_x"  : "DIR",
        "bc_y"  : "PER",
        "rows": 1,
        "cols": 2,
        "width": 1000,
        "height": 500
}

PATH_FELTOR = '../FELTOR/'
FELTOR_file = 'convection_hpc'

To run this simulations a simple class has been created in the model `FELTOR_class.py`, `FELTOR_Cpp` which is capable to create and modify the input file and run the code easily. The model has an example initial conditions, which would run a HW model with average. Let's make some examples.

Let's run the program for an interchange model:

In [3]:
FELTOR_IC = FELTOR_Cpp(PATH_FELTOR, FELTOR_file, 'input_IC.json',
                    'output_IC.nc', data = None)

FELTOR_IC.data['model'] = 'IC'
FELTOR_IC.save_input()
!{FELTOR_IC.run()}

{
	"Nx" : 120,
	"Nx_out" : 60,
	"Ny" : 120,
	"Ny_out" : 60,
	"adiabatic" : 0.0001,
	"amplitude" : 0.5,
	"bc_x" : "DIR",
	"bc_y" : "PER",
	"cols" : 2,
	"curvature" : 0.014999999999999999,
	"dens_prof" : -0.014999999999999999,
	"dt" : 0.01,
	"eps_pol" : 9.9999999999999995e-07,
	"eps_time" : 1e-10,
	"height" : 500,
	"itstp" : 10,
	"lx" : 200,
	"ly" : 200,
	"make_average" : 1,
	"maxout" : 80,
	"model" : "IC",
	"n" : 3,
	"n_out" : 3,
	"nu_perp" : 0.0050000000000000001,
	"posX" : 0.5,
	"posY" : 0.5,
	"rows" : 1,
	"sigma" : 10,
	"stages" : 3,
	"width" : 1000
}
The model we are using is IC
Add the average: 1
Physical parameters are: 
    Viscosity:       = 0.005
    Curvature:       = 0.015
  density profile:   = -0.015
 adiabatic parameter = 0.0001
Blob parameters are: 
    width:        10
    amplitude:    0.5
    posX:         0.5
    posY:         0.5
Boundary parameters are: 
    lx = 200
    ly = 200
Boundary conditions in x are: 
    DIRICHLET
Boundary conditions in y are: 
    PERIODI

For a HW model without average

In [4]:
FELTOR_HW_0 = FELTOR_Cpp(PATH_FELTOR, FELTOR_file, 'input_HW_0.json',
                    'output_HW_0.nc', data = None)

FELTOR_HW_0.data['make_average'] = 0
FELTOR_HW_0.data['model'] = "HW"
FELTOR_HW_0.save_input()
!{FELTOR_HW_0.run()}

{
	"Nx" : 120,
	"Nx_out" : 60,
	"Ny" : 120,
	"Ny_out" : 60,
	"adiabatic" : 0.0001,
	"amplitude" : 0.5,
	"bc_x" : "DIR",
	"bc_y" : "PER",
	"cols" : 2,
	"curvature" : 0.014999999999999999,
	"dens_prof" : -0.014999999999999999,
	"dt" : 0.01,
	"eps_pol" : 9.9999999999999995e-07,
	"eps_time" : 1e-10,
	"height" : 500,
	"itstp" : 10,
	"lx" : 200,
	"ly" : 200,
	"make_average" : 0,
	"maxout" : 80,
	"model" : "HW",
	"n" : 3,
	"n_out" : 3,
	"nu_perp" : 0.0050000000000000001,
	"posX" : 0.5,
	"posY" : 0.5,
	"rows" : 1,
	"sigma" : 10,
	"stages" : 3,
	"width" : 1000
}
The model we are using is HW
Add the average: 0
Physical parameters are: 
    Viscosity:       = 0.005
    Curvature:       = 0.015
  density profile:   = -0.015
 adiabatic parameter = 0.0001
Blob parameters are: 
    width:        10
    amplitude:    0.5
    posX:         0.5
    posY:         0.5
Boundary parameters are: 
    lx = 200
    ly = 200
Boundary conditions in x are: 
    DIRICHLET
Boundary conditions in y are: 
    PERIODI

For a HW model with average

In [ ]:
FELTOR_HW_1 = FELTOR_Cpp(PATH_FELTOR, FELTOR_file, 'input_HW_1.json',
                    'output_HW_1.nc', data = None)

FELTOR_HW_1.data['make_average'] = 1
FELTOR_HW_1.save_input()
!{FELTOR_HW_1.run()}

{
	"Nx" : 120,
	"Nx_out" : 60,
	"Ny" : 120,
	"Ny_out" : 60,
	"adiabatic" : 0.0001,
	"amplitude" : 0.5,
	"bc_x" : "DIR",
	"bc_y" : "PER",
	"cols" : 2,
	"curvature" : 0.014999999999999999,
	"dens_prof" : -0.014999999999999999,
	"dt" : 0.01,
	"eps_pol" : 9.9999999999999995e-07,
	"eps_time" : 1e-10,
	"height" : 500,
	"itstp" : 10,
	"lx" : 200,
	"ly" : 200,
	"make_average" : 1,
	"maxout" : 80,
	"model" : "HW",
	"n" : 3,
	"n_out" : 3,
	"nu_perp" : 0.0050000000000000001,
	"posX" : 0.5,
	"posY" : 0.5,
	"rows" : 1,
	"sigma" : 10,
	"stages" : 3,
	"width" : 1000
}
The model we are using is HW
Add the average: 1
Physical parameters are: 
    Viscosity:       = 0.005
    Curvature:       = 0.015
  density profile:   = -0.015
 adiabatic parameter = 0.0001
Blob parameters are: 
    width:        10
    amplitude:    0.5
    posX:         0.5
    posY:         0.5
Boundary parameters are: 
    lx = 200
    ly = 200
Boundary conditions in x are: 
    DIRICHLET
Boundary conditions in y are: 
    PERIODI

This model can be nicely convined with the analysis class `Analysis.Analyse`, which allows us to take the output file and analyse the solution.

In [ ]:
import sys
sys.path.insert(1, '../2D_FELTOR_Analysis/')

from Analysis import Analyse

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot        import axis, savefig, pcolormesh, imshow, show, figure, subplot, title, plot, colorbar

In [ ]:
File_name = 'output_IC.nc'
Analitics = Analyse(File_name)
Mass = Analitics.integrate('ions')
Potential = Analitics.integrate('potential')

In [ ]:
ftsz_title = 17
ftsz_label = 15

point = 10
X, Y = np.meshgrid(Analitics.x, Analitics.y)

figure(figsize = (24, 16))

subplot(2, 3, 1);
plot(Analitics.X_CM, Analitics.Y_CM, 'o'); title('Center of mass', fontsize = ftsz_title)
plot(Analitics.X_CM[point], Analitics.Y_CM[point], 'o', color = 'red')
plt.xlabel('CM X axis', fontsize = ftsz_label)
plt.ylabel('CM Y axis', fontsize = ftsz_label)


subplot(2, 3, 2); 
plot(Analitics.time, (Mass[0] - Mass) / Mass[0]); title('Error in total Mass', fontsize = ftsz_title)
plot(Analitics.time[point], (Mass[0] - Mass[point]) / Mass[0], 'o', color = 'red')
plt.xlabel('Time', fontsize = ftsz_label)
plt.ylabel('Error in total Mass', fontsize = ftsz_label)
plt.xscale('log')


subplot(2, 3, 3); 
plot(Analitics.V_CM_x, Analitics.V_CM_y, 'o'); title('Velocity of the CM', fontsize = ftsz_title)
plot(Analitics.V_CM_x[point], Analitics.V_CM_y[point], 'o', color = 'red')
plt.xlabel('V_CM X axis', fontsize = ftsz_label)
plt.ylabel('V_CM Y axis', fontsize = ftsz_label)


subplot(2, 3, 4)
plot(Analitics.time, Potential); title('Total Potential', fontsize = ftsz_title)
plot(Analitics.time[point], Potential[point], 'o', color = 'red')
plt.xlabel('Time', fontsize = ftsz_label)
plt.ylabel('Total Potential', fontsize = ftsz_label)
plt.xscale('log')


subplot(2, 3, 5)
pcolormesh(Analitics.x, Analitics.y, Analitics.ions[point], cmap = 'plasma', shading = 'gouraud'); title('Ions density', fontsize = ftsz_title)

subplot(2, 3, 6)
pcolormesh(Analitics.x, Analitics.y, Analitics.Data['vorticity'][point], cmap = 'plasma', shading = 'gouraud'); title('Vorticity', fontsize = ftsz_title)

show()